In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
%cd /content/gdrive/My Drive/Contelligenz Data Science/OTT Recommender System/Data/Raw Data

/content/gdrive/.shortcut-targets-by-id/1zUX2cs_LUVZEUVWeXs1TjALxiQz0Xuze/OTT Recommender System/Data/Raw Data


In [ ]:
!ls

user_events_raw.pkl


In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
import datetime as dt

In [ ]:
df = pickle.load(open('user_events_raw.pkl','rb'))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2099002 entries, 0 to 18770
Data columns (total 11 columns):
 #   Column                Dtype         
---  ------                -----         
 0   streamid              object        
 1   contentid             object        
 2   userid                object        
 3   subscription_plan_id  object        
 4   watchtime             datetime64[ns]
 5   duration              int64         
 6   browser               object        
 7   devicename            object        
 8   country               object        
 9   cityname              object        
 10  subdivisionname       object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 192.2+ MB


In [ ]:
df.head()

,streamid,contentid,userid,subscription_plan_id,watchtime,duration,browser,devicename,country,cityname,subdivisionname
0,04d0a064-b96b-3f49-a8f3-d1f692a9fbb7,6e119257-4599-45f5-878f-2b6ab98ac480,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-25 12:33:06,570,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
1,edc77967-30a3-3155-87da-141b6441b0d5,ab6c94cd-8000-419e-995c-f97f07ac6ce5,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 10:37:38,360,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
2,9ef047ac-c370-3406-9d8f-1282a82f9157,948927a5-7e54-49f9-9abb-79f1e749acfc,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 00:09:41,360,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
3,652eaa8e-1b83-305c-a2b7-9dc2c81cd348,948927a5-7e54-49f9-9abb-79f1e749acfc,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 00:00:11,420,,Samsung SM-E625F,IN,Hyderabad,Telangana
4,39aefc2e-0fff-32bb-872d-b33fa0d9a6c2,9b79eb7c-6ca9-4f0a-b1cb-c4afa4ffdffb,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-23 23:36:21,240,,Samsung SM-E625F,IN,Hyderabad,Telangana


In [ ]:
len(df.userid.unique())

71124

In [ ]:
df_user_item_interactions = df[['userid', 'contentid', 'streamid', 'watchtime', 'duration']]

In [ ]:
df_user_item_interactions.head()

,userid,contentid,streamid,watchtime,duration
0,edce2372-1a65-4333-984c-5d5d831e622d,6e119257-4599-45f5-878f-2b6ab98ac480,04d0a064-b96b-3f49-a8f3-d1f692a9fbb7,2021-05-25 12:33:06,570
1,edce2372-1a65-4333-984c-5d5d831e622d,ab6c94cd-8000-419e-995c-f97f07ac6ce5,edc77967-30a3-3155-87da-141b6441b0d5,2021-05-24 10:37:38,360
2,edce2372-1a65-4333-984c-5d5d831e622d,948927a5-7e54-49f9-9abb-79f1e749acfc,9ef047ac-c370-3406-9d8f-1282a82f9157,2021-05-24 00:09:41,360
3,edce2372-1a65-4333-984c-5d5d831e622d,948927a5-7e54-49f9-9abb-79f1e749acfc,652eaa8e-1b83-305c-a2b7-9dc2c81cd348,2021-05-24 00:00:11,420
4,edce2372-1a65-4333-984c-5d5d831e622d,9b79eb7c-6ca9-4f0a-b1cb-c4afa4ffdffb,39aefc2e-0fff-32bb-872d-b33fa0d9a6c2,2021-05-23 23:36:21,240


In [ ]:
len(df_user_item_interactions.userid.unique())

71124

In [ ]:
#df_user_item_interactions_grouped = df_user_item_interactions.groupby(['userid', 'contentid'])

In [ ]:
#df_user_item_interactions_grouped.head()

In [ ]:
%cd /content/gdrive/My Drive/Contelligenz Data Science/OTT Recommender System/Data/Processed Data

/content/gdrive/.shortcut-targets-by-id/1zUX2cs_LUVZEUVWeXs1TjALxiQz0Xuze/OTT Recommender System/Data/Processed Data


In [ ]:
df2 = pd.read_csv('content_catalog_v3.csv')

In [ ]:
df2.head()

,title,Thumbnail 2X3 URL,runtime,free,parental_rating,Genre,Genre2,Available Geo,publish_date,content_type,total_views,Genres,Key Elements,Cast Popularity,Mood,Audience appeal,Content Exclusivity,Content Language,Is dubbed ?,Original Language,Original industry,Wikipedia URL,IMDB URL,IMDB ID,Original Title,Runtime IMDB,Year,IMDB Rating,IMDB Votes,Plot,Actors,Director,Poster,Release Date,Awards,Award Winning,video_id,video
0,11Th Hour,https://snagfilms-a.akamaihd.net/38c1e2aa-64c1...,11596,False,18+,Drama,NaN,India,08-Apr-21,Originals,9776,"Drama, Thriller, Crime","Performances, Plot, Direction, Screenplay",Very Popular,"Exciting , Suspenseful",All audience,Exclusive,Telugu,No,Telugu,Tollywood,https://en.wikipedia.org/wiki/11th_Hour_(web_s...,https://www.imdb.com/title/tt13429172,tt13429172,11th Hour,NaN,2021–,6.3,294.0,"Courageous, fearless, and ambitious - She's a ...","Tamannaah Bhatia, Richard Bhakti Klein, Rajiv ...",NaN,https://m.media-amazon.com/images/M/MV5BNmYwNj...,08 April 2021,NaN,No,1bf00638-81f4-456b-8d51-e36387a7f7a3,39
1,Y,https://snagfilms-a.akamaihd.net/38c1e2aa-64c1...,5480,False,13+,Drama,NaN,India,01-Apr-21,Movie,4594,"Thriller, Suspense","Plot, Screenplay, Direction, Plot",Popular,Exciting,All audience,Exclusive,Telugu,No,Telugu,Tollywood,0,https://www.imdb.com/title/tt14334034,tt14334034,Y,91 min,2021,8.5,85.0,When a film director is on the lookout for a s...,"Raghu Babu, Rahul Ramakrishna, Srikanth, Gemin...",Baalu Adusumilli,https://m.media-amazon.com/images/M/MV5BMzliYj...,02 April 2021,NaN,No,a99c674d-bd5d-4bc5-878f-702bc53ab58d,216
2,No 1 Yaari With Rana,https://snagfilms-a.akamaihd.net/38c1e2aa-64c1...,20310,False,NR,Talk show,NaN,India,28-Mar-21,Originals,9864,Talk-show,"Interaction, Entertainment",Very Popular,Entertaining,All audience,Exclusive,Telugu,No,Telugu,Tollywood,https://en.wikipedia.org/wiki/No._1_Yaari,https://www.imdb.com/title/tt9126600,tt9126600,McDowell's No.1 Yaari with Rana,44 min,2017–2021,7.2,27.0,No.1 Yaari with Rana Season 2 is a chat show w...,"Nag Ashwin, Vijay Deverakonda, Rana Daggubati",NaN,https://m.media-amazon.com/images/M/MV5BYzliNj...,28 March 2021,NaN,No,a15df463-f33c-4b5c-b85c-392f9f5093d7,208
3,Thellavarithe Guruvaram,https://snagfilms-a.akamaihd.net/38c1e2aa-64c1...,7088,False,13+,Romance,Comedy,India,15-Apr-21,Movie,676,"Comedy, Romance","Plot, Screenplay, Direction, Entertainment",Popular,"Funny , Feel Good , Exciting",All audience,Exclusive,Telugu,No,Telugu,Tollywood,https://en.wikipedia.org/wiki/Thellavarithe_Gu...,https://www.imdb.com/title/tt13843858,tt13843858,Thellavarithe Guruvaram,120 min,2021,6.9,390.0,Mr. Simha's character is on the verge of getti...,"Ajay, Harsha Chemudu, Vajja Venkata Giridhar",Manikanth Gelli,https://m.media-amazon.com/images/M/MV5BYWNhMD...,27 March 2021,NaN,No,1685b28d-1966-4d5a-8119-e8379d0f40a2,32
4,Chaavu Kaburu Challaga,https://snagfilms-a.akamaihd.net/38c1e2aa-64c1...,7576,False,13+,Drama,NaN,India,22-Apr-21,Movie,21805,"Comedy, Romance","Plot, Plot, Direction",Very Popular,"Funny , Emotional , Feel Good",All audience,Exclusive,Telugu,No,Telugu,Tollywood,https://en.wikipedia.org/wiki/Chaavu_Kaburu_Ch...,https://www.imdb.com/title/tt13130198,tt13130198,Chaavu Kaburu Challaga,137 min,2021,6.0,631.0,A mortuary van driver experiences love at firs...,"Kartikeya Gummakonda, Lavanya Tripathi, Aamani",Koushik Pegallapati,https://m.media-amazon.com/images/M/MV5BMGU2M2...,19 March 2021,NaN,No,132237d6-404f-48cd-abce-296e7a3d4f1e,27


In [ ]:
df2 = df2.rename(columns = {"video_id":"contentid"})

In [ ]:
df3 = pd.merge(df_user_item_interactions, df2[['contentid', 'runtime']], on = 'contentid', how = 'left')

In [ ]:
df3.head()

,userid,contentid,streamid,watchtime,duration,runtime
0,edce2372-1a65-4333-984c-5d5d831e622d,6e119257-4599-45f5-878f-2b6ab98ac480,04d0a064-b96b-3f49-a8f3-d1f692a9fbb7,2021-05-25 12:33:06,570,NaN
1,edce2372-1a65-4333-984c-5d5d831e622d,ab6c94cd-8000-419e-995c-f97f07ac6ce5,edc77967-30a3-3155-87da-141b6441b0d5,2021-05-24 10:37:38,360,NaN
2,edce2372-1a65-4333-984c-5d5d831e622d,948927a5-7e54-49f9-9abb-79f1e749acfc,9ef047ac-c370-3406-9d8f-1282a82f9157,2021-05-24 00:09:41,360,7525.0
3,edce2372-1a65-4333-984c-5d5d831e622d,948927a5-7e54-49f9-9abb-79f1e749acfc,652eaa8e-1b83-305c-a2b7-9dc2c81cd348,2021-05-24 00:00:11,420,7525.0
4,edce2372-1a65-4333-984c-5d5d831e622d,9b79eb7c-6ca9-4f0a-b1cb-c4afa4ffdffb,39aefc2e-0fff-32bb-872d-b33fa0d9a6c2,2021-05-23 23:36:21,240,9204.0


In [ ]:
#df_user_item_interactions_grouped = df3.groupby(['userid', 'contentid', 'runtime'])['duration'].agg('sum').reset_index()

In [ ]:
df_user_item_interactions_grouped = df3.groupby(['userid', 'contentid']).agg({'duration':'sum', 'runtime':'first'})

In [ ]:
df_user_item_interactions_grouped.head(10)

duration  runtime
userid                                             contentid                                              
ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df9937... 1dd5c35f-015a-4529-9d60-8d185b3f6d9f       510   9068.0
                                                   f82ec50b-c45f-49bb-8bac-45d6c521e58e       600      NaN
ebbb8da9-12e9-4826-9d5f-788f7bf2a76e               7eb9b774-9256-4e1b-88a7-22ab39f8e1ad      8550   8639.0
                                                   918ca200-8d2a-4236-a825-9ba26d75d922         0   8505.0
                                                   91cb3608-d0ff-4d0b-9849-528f2003d214        60      NaN
                                                   f9e7f01d-27c5-4ca0-8c9d-363b12554c12        90      NaN
ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c46... 1685b28d-1966-4d5a-8119-e8379d0f40a2         0   7088.0
                                                   2f0b3711-2028-4987-b4fb-d60ce3631779      2070      NaN
                                                   326d790b-8e0b-4639-8129-53d22a740bfe      6870   7369.0
                                                   43538392-b0eb-43cb-aa99-f2cca8e44e53        90      NaN

In [ ]:
df_user_item_interactions_grouped['view_percentage'] = df_user_item_interactions_grouped['duration'] / df_user_item_interactions_grouped['runtime']

In [ ]:
len(df_user_item_interactions_grouped.runtime)

907471

In [ ]:
df_user_item_interactions_grouped.runtime.isna().sum()

642832

In [ ]:
df_user_item_interactions_grouped.head()

duration  ...  view_percentage
userid                                             contentid                                       ...                 
ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df9937... 1dd5c35f-015a-4529-9d60-8d185b3f6d9f       510  ...         0.056242
                                                   f82ec50b-c45f-49bb-8bac-45d6c521e58e       600  ...              NaN
ebbb8da9-12e9-4826-9d5f-788f7bf2a76e               7eb9b774-9256-4e1b-88a7-22ab39f8e1ad      8550  ...         0.989698
                                                   918ca200-8d2a-4236-a825-9ba26d75d922         0  ...         0.000000
                                                   91cb3608-d0ff-4d0b-9849-528f2003d214        60  ...              NaN

[5 rows x 3 columns]

In [ ]:
df_user_item_interactions_grouped.loc[df_user_item_interactions_grouped['view_percentage'] > 1]

duration  ...  view_percentage
userid                                             contentid                                       ...                 
ebbbe22e-0936-45dc-91eb-d67d0e9f3a90               918ca200-8d2a-4236-a825-9ba26d75d922      8640  ...         1.015873
                                                   b1f55aba-dcae-4035-9549-0461b796cbba     11400  ...         1.284797
ebc0d993-af9c-4932-878e-ed8c15e860b2               608182fd-24b7-4558-a6a5-e82a4502e64a     36810  ...         4.150412
ebc12aa4ca618e10222e5e4921389034cbbc17d9b6d307c... 8fd49f67-4871-4ffd-9f66-584f10f39501     18900  ...         2.957746
ebc6e03c-9ed7-4b63-bb8b-1441286b6a51               132237d6-404f-48cd-abce-296e7a3d4f1e      8430  ...         1.112724
...                                                                                           ...  ...              ...
ffffe6b1-b546-4542-85c9-a7601f45c440               86812def-4ddb-4018-af9d-50204a37249d      7410  ...         1.008575
                                                   a99c674d-bd5d-4bc5-878f-702bc53ab58d     16530  ...         3.016423
                                                   b59ed222-f58c-4ceb-ba62-8df4cd110df8      8670  ...         1.008140
fffff8bc-a107-4ed6-a6e5-4e505a0e5952               132237d6-404f-48cd-abce-296e7a3d4f1e     32940  ...         4.347941
                                                   1685b28d-1966-4d5a-8119-e8379d0f40a2     31470  ...         4.439898

[32081 rows x 3 columns]

In [ ]:
df_user_item_interactions_grouped.reset_index()

,userid,contentid,duration,runtime,view_percentage
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,1dd5c35f-015a-4529-9d60-8d185b3f6d9f,510,9068.0,0.056242
1,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,f82ec50b-c45f-49bb-8bac-45d6c521e58e,600,NaN,NaN
2,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,7eb9b774-9256-4e1b-88a7-22ab39f8e1ad,8550,8639.0,0.989698
3,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,918ca200-8d2a-4236-a825-9ba26d75d922,0,8505.0,0.000000
4,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,91cb3608-d0ff-4d0b-9849-528f2003d214,60,NaN,NaN
...,...,...,...,...,...
907466,fffff8bc-a107-4ed6-a6e5-4e505a0e5952,281208be-e658-4ed5-8c44-f32d038015fa,1170,NaN,NaN
907467,fffff8bc-a107-4ed6-a6e5-4e505a0e5952,2db4f7e9-ea53-45aa-8733-03e78e9b478a,1740,NaN,NaN
907468,fffff8bc-a107-4ed6-a6e5-4e505a0e5952,681af8ca-4f02-4daa-aa17-8a95c7510adb,210,8224.0,0.025535
907469,fffff8bc-a107-4ed6-a6e5-4e505a0e5952,8d9d0252-94ac-475b-88d2-a22d83e07c37,0,NaN,NaN


# **User Profile Data Processing**

---


In [ ]:
df.head()

,streamid,contentid,userid,subscription_plan_id,watchtime,duration,browser,devicename,country,cityname,subdivisionname
0,04d0a064-b96b-3f49-a8f3-d1f692a9fbb7,6e119257-4599-45f5-878f-2b6ab98ac480,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-25 12:33:06,570,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
1,edc77967-30a3-3155-87da-141b6441b0d5,ab6c94cd-8000-419e-995c-f97f07ac6ce5,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 10:37:38,360,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
2,9ef047ac-c370-3406-9d8f-1282a82f9157,948927a5-7e54-49f9-9abb-79f1e749acfc,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 00:09:41,360,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
3,652eaa8e-1b83-305c-a2b7-9dc2c81cd348,948927a5-7e54-49f9-9abb-79f1e749acfc,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 00:00:11,420,,Samsung SM-E625F,IN,Hyderabad,Telangana
4,39aefc2e-0fff-32bb-872d-b33fa0d9a6c2,9b79eb7c-6ca9-4f0a-b1cb-c4afa4ffdffb,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-23 23:36:21,240,,Samsung SM-E625F,IN,Hyderabad,Telangana


In [ ]:
df_up = df[['userid', 'devicename', 'cityname', 'watchtime', 'contentid']]

In [ ]:
df_up.head()

,userid,devicename,cityname,watchtime,contentid
0,edce2372-1a65-4333-984c-5d5d831e622d,Samsung SM-E625F,Tanuku,2021-05-25 12:33:06,6e119257-4599-45f5-878f-2b6ab98ac480
1,edce2372-1a65-4333-984c-5d5d831e622d,Samsung SM-E625F,Tanuku,2021-05-24 10:37:38,ab6c94cd-8000-419e-995c-f97f07ac6ce5
2,edce2372-1a65-4333-984c-5d5d831e622d,Samsung SM-E625F,Tanuku,2021-05-24 00:09:41,948927a5-7e54-49f9-9abb-79f1e749acfc
3,edce2372-1a65-4333-984c-5d5d831e622d,Samsung SM-E625F,Hyderabad,2021-05-24 00:00:11,948927a5-7e54-49f9-9abb-79f1e749acfc
4,edce2372-1a65-4333-984c-5d5d831e622d,Samsung SM-E625F,Hyderabad,2021-05-23 23:36:21,9b79eb7c-6ca9-4f0a-b1cb-c4afa4ffdffb


In [ ]:
df_up = df_up.groupby('userid').agg({'devicename': lambda x: list(set(x)), 'cityname': lambda x: list(set(x)), 'contentid': lambda y: list(set(y)), 'watchtime': 'first'}).reset_index()

In [ ]:
df_up.head()

,userid,devicename,cityname,contentid,watchtime
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,"[, vivo 1919]",[Hyderabad],"[1dd5c35f-015a-4529-9d60-8d185b3f6d9f, f82ec50...",2021-05-03 04:40:58
1,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,[Generic Smartphone],[Hyderabad],"[7eb9b774-9256-4e1b-88a7-22ab39f8e1ad, 918ca20...",2021-05-20 13:39:49
2,ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c4...,"[, Generic Smartphone, vivo 1802, Apple iPhone...","[Bengaluru, Mumbai, Chittoor, Nellore, Tirupat...","[ab6c94cd-8000-419e-995c-f97f07ac6ce5, 326d790...",2021-05-23 00:08:28
3,ebbbe22e-0936-45dc-91eb-d67d0e9f3a90,"[, Generic Smartphone, Generic_Android AFTT]",[Hyderabad],"[4620b795-ef41-493d-888e-a7eca2c51800, 7b99a5d...",2021-05-24 08:01:52
4,ebbc6236-c635-4244-a5b7-0ccfcda54056,"[, Generic Smartphone, Generic_Android BRAVIA ...","[Hyderabad, Karimnagar]","[d66d97b2-2cbb-401a-8e90-65ef366b5e6e, 69881c5...",2021-05-23 10:58:48


In [ ]:
df_up.devicename = df_up.devicename.apply(lambda x: list(filter(None, x)))

In [ ]:
df_up.head()

,userid,devicename,cityname,contentid,watchtime
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,[vivo 1919],[Hyderabad],"[1dd5c35f-015a-4529-9d60-8d185b3f6d9f, f82ec50...",2021-05-03 04:40:58
1,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,[Generic Smartphone],[Hyderabad],"[7eb9b774-9256-4e1b-88a7-22ab39f8e1ad, 918ca20...",2021-05-20 13:39:49
2,ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c4...,"[Generic Smartphone, vivo 1802, Apple iPhone11,2]","[Bengaluru, Mumbai, Chittoor, Nellore, Tirupat...","[ab6c94cd-8000-419e-995c-f97f07ac6ce5, 326d790...",2021-05-23 00:08:28
3,ebbbe22e-0936-45dc-91eb-d67d0e9f3a90,"[Generic Smartphone, Generic_Android AFTT]",[Hyderabad],"[4620b795-ef41-493d-888e-a7eca2c51800, 7b99a5d...",2021-05-24 08:01:52
4,ebbc6236-c635-4244-a5b7-0ccfcda54056,"[Generic Smartphone, Generic_Android BRAVIA 4K...","[Hyderabad, Karimnagar]","[d66d97b2-2cbb-401a-8e90-65ef366b5e6e, 69881c5...",2021-05-23 10:58:48


In [ ]:
df_up['deviceCount'] = df_up['devicename'].str.len()

In [ ]:
df_up['citiesCount'] = df_up['cityname'].str.len()

In [ ]:
df_up = df_up.rename(columns = {'contentid': 'watchHistory', 'devicename': 'devices', 'cityname': 'cities'})

In [ ]:
df_up['titlesCount'] = df_up['watchHistory'].str.len()

In [ ]:
df_dt = df[['userid', 'watchtime', 'duration']]

In [ ]:
df_dt = df_dt.groupby(['userid']).agg({'watchtime':['min', 'max', 'count'], 'duration':['sum']}).reset_index()

In [ ]:
df_dt.columns = ['_'.join(col) for col in df_dt.columns.values]

In [ ]:
df_dt.head()

,userid_,watchtime_min,watchtime_max,watchtime_count,duration_sum
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,2021-05-02 01:24:51,2021-05-03 04:40:58,6,1110
1,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,2021-05-08 06:09:07,2021-05-20 13:39:49,6,8700
2,ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c4...,2021-05-01 02:21:31,2021-05-23 00:08:28,22,64980
3,ebbbe22e-0936-45dc-91eb-d67d0e9f3a90,2021-05-01 04:52:36,2021-05-24 08:01:52,74,79590
4,ebbc6236-c635-4244-a5b7-0ccfcda54056,2021-05-08 11:24:17,2021-05-23 10:58:48,34,26640


In [ ]:
df_dt = df_dt.rename(columns={'userid_':'userid', 'watchtime_min': 'startDate', 'watchtime_max': 'lastDate', 'watchtime_count': 'sessionsCount', 'duration_sum': 'totalDuration'})

In [ ]:
df_up = pd.merge(df_up, df_dt, on = ['userid'], how = 'left')

In [ ]:
df_up['startDate'] = df_up['startDate'].dt.date
df_up['lastDate'] = df_up['lastDate'].dt.date

In [ ]:
df_up.head()

,userid,devices,cities,watchHistory,watchtime,deviceCount,citiesCount,titlesCount,startDate,lastDate,sessionsCount,totalDuration
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,[vivo 1919],[Hyderabad],"[1dd5c35f-015a-4529-9d60-8d185b3f6d9f, f82ec50...",2021-05-03 04:40:58,1,1,2,2021-05-02,2021-05-03,6,1110
1,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,[Generic Smartphone],[Hyderabad],"[7eb9b774-9256-4e1b-88a7-22ab39f8e1ad, 918ca20...",2021-05-20 13:39:49,1,1,4,2021-05-08,2021-05-20,6,8700
2,ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c4...,"[Generic Smartphone, vivo 1802, Apple iPhone11,2]","[Bengaluru, Mumbai, Chittoor, Nellore, Tirupat...","[ab6c94cd-8000-419e-995c-f97f07ac6ce5, 326d790...",2021-05-23 00:08:28,3,6,10,2021-05-01,2021-05-23,22,64980
3,ebbbe22e-0936-45dc-91eb-d67d0e9f3a90,"[Generic Smartphone, Generic_Android AFTT]",[Hyderabad],"[4620b795-ef41-493d-888e-a7eca2c51800, 7b99a5d...",2021-05-24 08:01:52,2,1,28,2021-05-01,2021-05-24,74,79590
4,ebbc6236-c635-4244-a5b7-0ccfcda54056,"[Generic Smartphone, Generic_Android BRAVIA 4K...","[Hyderabad, Karimnagar]","[d66d97b2-2cbb-401a-8e90-65ef366b5e6e, 69881c5...",2021-05-23 10:58:48,2,2,24,2021-05-08,2021-05-23,34,26640


In [ ]:
############## Active Duration = lastDate - startDate ##########
df_up['activeDuration'] = df_up['lastDate'] - df_up['startDate'] + dt.timedelta(days = 1)

In [ ]:
############ Average Number of session per active day ###########
df_up['avgSessionsPerDay'] = df_up['sessionsCount'] / df_up['activeDuration'].dt.days

In [ ]:
df_up.drop('watchtime', axis = 1, inplace = True)

In [ ]:
df_up.head(6)

,userid,devices,cities,watchHistory,deviceCount,citiesCount,titlesCount,startDate,lastDate,sessionsCount,totalDuration,activeDuration,avgSessionsPerDay
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,[vivo 1919],[Hyderabad],"[1dd5c35f-015a-4529-9d60-8d185b3f6d9f, f82ec50...",1,1,2,2021-05-02,2021-05-03,6,1110,2 days,3.000000
1,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,[Generic Smartphone],[Hyderabad],"[7eb9b774-9256-4e1b-88a7-22ab39f8e1ad, 918ca20...",1,1,4,2021-05-08,2021-05-20,6,8700,13 days,0.461538
2,ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c4...,"[Generic Smartphone, vivo 1802, Apple iPhone11,2]","[Bengaluru, Mumbai, Chittoor, Nellore, Tirupat...","[ab6c94cd-8000-419e-995c-f97f07ac6ce5, 326d790...",3,6,10,2021-05-01,2021-05-23,22,64980,23 days,0.956522
3,ebbbe22e-0936-45dc-91eb-d67d0e9f3a90,"[Generic Smartphone, Generic_Android AFTT]",[Hyderabad],"[4620b795-ef41-493d-888e-a7eca2c51800, 7b99a5d...",2,1,28,2021-05-01,2021-05-24,74,79590,24 days,3.083333
4,ebbc6236-c635-4244-a5b7-0ccfcda54056,"[Generic Smartphone, Generic_Android BRAVIA 4K...","[Hyderabad, Karimnagar]","[d66d97b2-2cbb-401a-8e90-65ef366b5e6e, 69881c5...",2,2,24,2021-05-08,2021-05-23,34,26640,16 days,2.125000
5,ebbe0a98f9afb8168f1b6ea8c1530068f81c11a620e86c...,[XiaoMi Redmi Note 7 Pro],[Hyderabad],"[520880cd-4ae5-4f30-a7f0-0c8e8052c6fe, 891e5cf...",1,1,2,2021-05-18,2021-05-18,2,60,1 days,2.000000


In [ ]:
max(df_up.sessionsCount)

607

In [ ]:
#df_up.to_csv('/content/gdrive/My Drive/Contelligenz Data Science/OTT Recommender System/Data/Processed Data/New Data/user_profiles.csv', encoding = 'utf-8')

# **To be Executed** 

In [ ]:
df_user_profile = pd.read_csv('user_profile_v1.csv', converters={'devices':eval, 'viewing_history':eval})

In [ ]:
df_user_profile.head()

,userid,devices,deviceCount,viewing_history,total_titles_viewed,citiesList,citiesCount,startDate,latestDate,activeDuration,sessionsCount,sessionsperDay,total_view_minutes,subscription_type,userAge,Recency,userType,userStatus
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,[vivo 1919],1,[1dd5c35f-015a-4529-9d60-8d185b3f6d9f],1,['Hyderabad'],1,2021-05-02,2021-05-02,1,1,1.000000,8.5,Paid,24,24,Existing,Inactive
1,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,[Generic Smartphone],1,[918ca200-8d2a-4236-a825-9ba26d75d9227eb9b774-...,2,['Hyderabad'],1,2021-05-08,2021-05-11,4,4,1.000000,142.5,Paid,18,15,Existing,Active
2,ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c4...,"[Apple iPhone11,2Generic Smartphone]",2,[326d790b-8e0b-4639-8129-53d22a740bfe918ca200-...,4,['Hyderabad' 'Bengaluru'],2,2021-05-01,2021-05-18,18,4,0.222222,302.5,Paid,25,8,Existing,Active
3,ebbbe22e-0936-45dc-91eb-d67d0e9f3a90,[Generic_Android AFTTGeneric Smartphone],3,[e95bcf76-9880-4dba-8554-058783e9d489d66d97b2-...,13,['Hyderabad'],1,2021-05-02,2021-05-23,22,31,1.409091,744.0,Paid,24,3,Existing,Active
4,ebbc6236-c635-4244-a5b7-0ccfcda54056,[Generic SmartphoneGeneric_Android BRAVIA 4K 2...,3,[d66d97b2-2cbb-401a-8e90-65ef366b5e6e391db145-...,14,['Hyderabad' 'Karimnagar'],2,2021-05-10,2021-05-21,12,23,1.916667,173.5,Paid,16,5,Existing,Active


In [ ]:
type(df_user_profile['citiesList'][0])

str

In [ ]:
df_user_profile['citiesList'] = df_user_profile['citiesList'].apply(lambda x: x.replace('[','').replace(']','')) 
df_user_profile['citiesList'] = df_user_profile['citiesList'].apply(lambda x: x.replace('"', '').replace('\'', ''))

In [ ]:
def split_it(val):
  return re.findall('[A-Z][^A-Z]*', val)

In [ ]:
df_user_profile['citiesList'] = df_user_profile['citiesList'].apply(split_it)

In [ ]:
df_user_profile.head()

,userid,devices,deviceCount,viewing_history,total_titles_viewed,citiesList,citiesCount,startDate,latestDate,activeDuration,sessionsCount,sessionsperDay,total_view_minutes,subscription_type,userAge,Recency,userType,userStatus
0,ebbb77a5e7d1b4083342281ee8b1509cdb486f3b2df993...,[vivo 1919],1,[1dd5c35f-015a-4529-9d60-8d185b3f6d9f],1,[Hyderabad],1,2021-05-02,2021-05-02,1,1,1.000000,8.5,Paid,24,24,Existing,Inactive
1,ebbb8da9-12e9-4826-9d5f-788f7bf2a76e,[Generic Smartphone],1,[918ca200-8d2a-4236-a825-9ba26d75d9227eb9b774-...,2,[Hyderabad],1,2021-05-08,2021-05-11,4,4,1.000000,142.5,Paid,18,15,Existing,Active
2,ebbba3b51cd791646e1bce9dfab33f41f9c0d742ef30c4...,"[Apple iPhone11,2Generic Smartphone]",2,[326d790b-8e0b-4639-8129-53d22a740bfe918ca200-...,4,"[Hyderabad , Bengaluru]",2,2021-05-01,2021-05-18,18,4,0.222222,302.5,Paid,25,8,Existing,Active
3,ebbbe22e-0936-45dc-91eb-d67d0e9f3a90,[Generic_Android AFTTGeneric Smartphone],3,[e95bcf76-9880-4dba-8554-058783e9d489d66d97b2-...,13,[Hyderabad],1,2021-05-02,2021-05-23,22,31,1.409091,744.0,Paid,24,3,Existing,Active
4,ebbc6236-c635-4244-a5b7-0ccfcda54056,[Generic SmartphoneGeneric_Android BRAVIA 4K 2...,3,[d66d97b2-2cbb-401a-8e90-65ef366b5e6e391db145-...,14,"[Hyderabad , Karimnagar]",2,2021-05-10,2021-05-21,12,23,1.916667,173.5,Paid,16,5,Existing,Active


In [ ]:
len(df_user_profile)

60911

In [ ]:
len(df_user_profile.userid.unique())

60911